In [1]:
import requests
import pandas as pd

In [2]:
# Information We Want
street = []
location = []
postcode = []
country = []
storeid = []
openingdate = []

# Function which return a value or null dependening on the availability of keys in the API Response
def safe_get(dataset, keys, default="N/A"):
    try:
        for key in keys:
            dataset = dataset[key]
        return dataset
    except (KeyError, TypeError):
        return default

# Collection total number of stores and other information from API response. (Limit assumed to be 20)
URL = 'https://prod-cdn.us.yextapis.com/v2/accounts/me/search/vertical/query?experienceKey=waitrose-locator-search&api_key=b89aa54de5943d5158a4d0b4062fe8d1&v=20220511&version=PRODUCTION&locale=en-GB&input=&verticalKey=locations&filters=%7B%22builtin.location%22%3A%7B%22%24near%22%3A%7B%22lat%22%3A51.230223%2C%22lng%22%3A-3.835141%2C%22radius%22%3A1000000%7D%7D%7D&offset=0&retrieveFacets=true&skipSpellCheck=false&sessionTrackingEnabled=false&sortBys=%5B%5D&source=STANDARD'
r = requests.get(url = URL)
data = r.json()
limit = 20
remainder = data['response']['resultsCount'] % limit
totalstores = data['response']['resultsCount']

# Function which extracts all information from API response
for i in range(0, totalstores + limit - remainder, limit):
    URL = f'https://prod-cdn.us.yextapis.com/v2/accounts/me/search/vertical/query?experienceKey=waitrose-locator-search&api_key=b89aa54de5943d5158a4d0b4062fe8d1&v=20220511&version=PRODUCTION&locale=en-GB&input=&verticalKey=locations&filters=%7B%22builtin.location%22%3A%7B%22%24near%22%3A%7B%22lat%22%3A51.230223%2C%22lng%22%3A-3.835141%2C%22radius%22%3A1000000%7D%7D%7D&offset={i}&retrieveFacets=true&skipSpellCheck=false&sessionTrackingEnabled=false&sortBys=%5B%5D&source=STANDARD'
    r = requests.get(url = URL)
    data = r.json()

    if i != (totalstores - remainder):
        for j in range(0, limit):
            street.append(safe_get(data, ['response', 'results', j, 'data', 'address', 'line1']))
            location.append(safe_get(data, ['response', 'results', j, 'data', 'address', 'city']))
            postcode.append(safe_get(data, ['response', 'results', j, 'data', 'address', 'postalCode']))
            country.append(safe_get(data, ['response', 'results', j, 'data', 'address', 'countryCode']))
            storeid.append(safe_get(data, ['response', 'results', j, 'data', 'id']))
            openingdate.append(safe_get(data, ['response', 'results', 0, 'data', 'yearEstablished']))

    elif i == (totalstores - remainder):
        for j in range(0, remainder):
            street.append(safe_get(data, ['response', 'results', j, 'data', 'address', 'line1']))
            location.append(safe_get(data, ['response', 'results', j, 'data', 'address', 'city']))
            postcode.append(safe_get(data, ['response', 'results', j, 'data', 'address', 'postalCode']))
            country.append(safe_get(data, ['response', 'results', j, 'data', 'address', 'countryCode']))
            storeid.append(safe_get(data, ['response', 'results', j, 'data', 'id']))
            openingdate.append(safe_get(data, ['response', 'results', 0, 'data', 'yearEstablished']))



In [3]:
waitrose = pd.DataFrame({'street': street, 'location': location,'postcode': postcode,'country': country,'storeid': storeid,'openingdate':openingdate})

In [5]:
waitrose.head()

,street,location,postcode,country,storeid,openingdate
0,Birds Lane,Cowbridge,CF71 7YP,GB,735,2012
1,Red Lion Court,Wellington,TA21 8RD,GB,315,2012
2,School Way,Okehampton,EX20 4LY,GB,234,2012
3,Croescadarn Close,Cardiff,CF23 8AN,GB,457,2012
4,Station Road,Holsworthy,EX22 6AZ,GB,480,2012
